<a href="https://colab.research.google.com/github/znatri/Biologically_Plausible_NN_Learning/blob/main/NMA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>